<a href="https://colab.research.google.com/github/ha-yun/GSTEC/blob/main/210823.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 웹 애플리케이션

subscribe의 model만들기  
Meta클래스는 모델폼을 사용하면서 사용..?  
Meta클래스는 외부적인 옵션을 지정해주는 unique_together  

model을 변경하였으니 터미널에서  
python manage.py makemigrations  
python manage.py migrate

In [ ]:
# subscribeapp/models.py
class Subscription(models.Model):
    user = models.ForeignKey(User, on_delete=models.CASCADE,
                             related_name='subscription', null=False)
    project = models.ForeignKey(Project, on_delete=models.CASCADE,
                                related_name='subscription', null=False)
  
    class Meta:
        unique_together = ['user','project']

subscription view

In [ ]:
# subscribeapp/views.py
@method_decorator(login_required, 'get')
class SubscriptionView(RedirectView):

    def get(self, request, *args, **kwargs):
        user = request.user
        project = Project.objects.get(pk=kwargs['project_pk'])
        subscription = Subscription.objects.filter(user=user,
                                                   project=project)
        if subscription.exists():
            subscription.delete()
        else:
            Subscription(user=user, project=project).save()

        return super().get(request, *args, **kwargs)

    def get_redirect_url(self, *args, **kwargs):
        return reverse('projectapp:detail', kwargs={'pk' : kwargs['project_pk']}) 


# subscribeapp/urls.py

app_name = 'subscribeapp'

urlpatterns=[
    path('subscribe/<int:project_pk>', SubscriptionView.as_view(), name='subscribe'),
] 

In [ ]:
# projectapp/templates/projectapp/detail.html
    <div class="text-center">
        <a href="{% url 'subscribeapp:subscribe' project_pk=target_project.pk %}"
            class="btn btn-primary rounded-pill px-5">
            Subscribe
        </a>
    </div>

In [ ]:
# projectapp/views.py
class ProjectDetailView(DetailView, MultipleObjectMixin):
    model = Project
    context_object_name = 'target_project'
    template_name = 'projectapp/detail.html'

    paginate_by = 20

    def get_context_data(self, **kwargs):
        user = self.request.user
        project = self.object

        if user.is_authenticated:
            subscription = Subscription.objects.filter(user=user,
                                                       project=project)
        else:
            subscription = None

        article_list = Article.objects.filter(project=self.object)
        return super().get_context_data(object_list=article_list,
                                        subscription=subscription,
                                        **kwargs)

In [ ]:
# projectapp/templates/projectapp/detail.html
    {% if user.is_authenticated %}
    <div class="text-center">
        {% if not subscription %}
        <a href="{% url 'subscribeapp:subscribe' project_pk=target_project.pk %}"
           class="btn btn-primary rounded-pill px-5">
            Subscribe
        </a>
        {% else %}
        <a href="{% url 'subscribeapp:subscribe' project_pk=target_project.pk %}"
           class="btn btn-secondary rounded-pill px-5">
            Unsubscribe
        </a>
        {% endif %}
    </div>
    {% endif %}

field lookup : project__in

In [ ]:
# subscribeapp/views.py

class SubscriptionListView(ListView):
    model = Article
    context_object_name = 'article_list'
    template_name = 'subscribeapp/list.html'

    paginate_by = 20

    def get_queryset(self):
        project_list = Subscription.objects.filter(user=self.request.user).values_list('project')
        article_list = Article.objects.filter(project__in=project_list)
        return article_list


# subscribeapp/urls.py
path('list/', SubscriptionListView.as_view(), name='list'),

In [ ]:
# subscribeapp/templates/subscribeapp/list.html
{% extends 'base.html' %}

{% block content %}

    <div>
        {% include 'snippets/list_fragment.html' %}
    </div>

{% endblock %} 


# templates/header.html
       <span>
            <a href="{% url 'subscribeapp:list' %}">
                Subscription
            </a>
        </span>

# 4. 머신러닝

In [ ]:
# 210819.ipynb